In [64]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.1
)
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000024009859EA0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000240098596C0>, root_client=<openai.OpenAI object at 0x000002400985A110>, root_async_client=<openai.AsyncOpenAI object at 0x0000024009859F60>, model_name='gpt-4o', temperature=0.1, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [65]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001"
)
embeddings

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000024009A0B910>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x000002400C644F70>, model='models/embedding-001', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

In [66]:
from langchain_community.document_loaders import DirectoryLoader,PyPDFLoader

pdf_directory = r"C:\Users\ParthaPratimKalita\document_portal\notebook\data_hw"

loader = PyPDFDirectoryLoader(
    path = r"C:\Users\ParthaPratimKalita\document_portal\notebook\data_hw",
    glob = "**/[!.]*.pdf",
    silent_errors = False,
    load_hidden = False,
    recursive = False,
    extract_images = False,
    password = None,
    mode = "page",
    # images_to_text = None,
    headers = None,
    extraction_mode = "plain",
    # extraction_kwargs = None,
)

documents = loader.load()

In [67]:
docs = await loader.aload()
print(docs[6].page_content[:100])
print(docs[6].metadata)

ACCOUNTING                               
BRENDA VIEUX                                              
{'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'keywords': '', 'source': 'C:\\Users\\ParthaPratimKalita\\document_portal\\notebook\\data_hw\\burgbcwhu_0 burgbhkqt_3122.pdf', 'total_pages': 3, 'page': 2, 'page_label': '3'}


In [68]:
len(documents)

193

In [69]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitters = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=150
)

chunks = text_splitters.split_documents(documents)
len(chunks)

1120

In [70]:
from langchain.vectorstores import FAISS

vector_store = FAISS.from_documents(chunks, embeddings)
vector_store

In [78]:
retriever = vector_store.as_retriever(search_kwargs={"k": 10})

retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002400984FC10>, search_kwargs={'k': 10})

In [72]:
from langchain.prompts import PromptTemplate


prompt_template= """
You are a helpful assistant. You will be given a question and a set of documents.
Your task is to answer the question based on the documents provided.

Context : {context}

Question : {question}

"""
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template
)
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nYou are a helpful assistant. You will be given a question and a set of documents.\nYour task is to answer the question based on the documents provided.\n\nContext : {context}\n\nQuestion : {question}\n\n')

In [73]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [74]:
def format_docs(docs):
    return "\n".join([doc.page_content for doc in docs])

In [75]:
rag_chain = (
    {"context" : retriever | format_docs, "question" : RunnablePassthrough()}
    | prompt
    | llm
    | parser
)

In [80]:
rag_chain.invoke("Tell me about the llama2 finetning benchmark experiments?")

'The Llama 2 fine-tuning benchmark experiments involve evaluating the performance of various Llama 2 models across a suite of popular academic and standard benchmarks. The models are compared based on their performance on these benchmarks, which are grouped into different categories. The results are summarized in tables that show the performance of different Llama 2 models, such as 7B, 13B, 34B, and 70B, across these benchmarks.\n\nFor instance, in the context provided, the Llama 2 models are evaluated on grouped academic benchmarks, with scores indicating their performance. The 70B model generally shows the highest performance across these benchmarks, followed by the 34B, 13B, and 7B models. The performance is measured in various metrics, such as pass rates for specific tasks like Human-Eval and MBPP.\n\nAdditionally, the fine-tuning experiments also compare Llama 2 models with other models like MPT and Falcon, showing how Llama 2 models perform relative to these other models on tasks

In [77]:
rag_chain.invoke("what is the Scaling trends for the reward models in Llama 2 models?")

'The scaling trends for the reward models in Llama 2 models involve fine-tuning different model sizes on an increasing amount of reward model data collected weekly. As more preference data is collected, the reward models improve, allowing for progressively better versions of Llama 2-Chat. The expected result is that larger models perform better with more data. Additionally, it is important to gather new preference data using the latest Llama 2-Chat iterations before a new tuning iteration to keep the reward model on-distribution and maintain accurate rewards.'